In [1]:
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

import rental_utils; reload(rental_utils)

%matplotlib inline
sns.set(style="whitegrid")

In [8]:
train_raw = rental_utils.clean(pd.read_json('Data/train.json'))
test_raw = rental_utils.clean(pd.read_json('Data/test.json'))

In [26]:
import re
import nltk
from bs4 import BeautifulSoup

In [21]:
from nltk.corpus import stopwords
print stopwords.words("english") 

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [30]:
example1 = BeautifulSoup(train_raw['description'].values[0], "html5lib")  
print(train_raw["description"].values[0])
print example1.get_text()

letters_only = re.sub("[^a-zA-Z]", " ", example1.get_text())
print(letters_only)

lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split() 

# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print(words)

A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here? Modern Designed Bathroom w/ a Deep Spa Soaking Tub? Room to Room AC/Heat? Real Oak Hardwood Floors? Rain Forest Shower Head? SS steel Appliances w/ Chef Gas Cook Oven & LG Fridge? washer /dryer in the apt? Cable Internet Ready? Granite Counter Top Kitchen w/ lot of cabinet storage spaceIt's Just A Few blocks To L Train<br /><br />Don't miss out!<br /><br />We have several great apartments in the immediate area.<br /><br />For additional information 687-878-2229<p><a  website_redacted 
A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here? Modern Designed Bathroom w/ a Deep Spa Soaking Tub? Room to Room AC/Heat? Real Oak Hardwood Floors? Rain Forest Shower Head? SS steel Appliances w/ Chef Gas Cook Oven & LG Fridge? washer /dryer in the apt? Cable Internet Ready? Granite Counter Top Kitchen w/ lot of cabinet storage spaceIt's Just A Few blocks To

In [36]:
def description_to_words( raw_description ):
    # Function to convert a raw description to a string of words
    # The input is a single string (a raw apartment description), and 
    # the output is a single string (a preprocessed description)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_description).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  

In [37]:
clean_description= description_to_words( train_raw["description"].values[0] )
print(clean_description)

brand new bedroom bath apartmentenjoy following apartment features rent modern designed bathroom w deep spa soaking tub room room ac heat real oak hardwood floors rain forest shower head ss steel appliances w chef gas cook oven lg fridge washer dryer apt cable internet ready granite counter top kitchen w lot cabinet storage spaceit blocks l traindon miss several great apartments immediate area additional information


In [41]:
num_descriptions = train_raw["description"].values.size
clean_train_descriptions = []
for i in xrange( 0, num_descriptions ):
    if( (i+1)%10000 == 0 ):
        print "Description %d of %d\n" % ( i+1, num_descriptions ) 
    clean_train_descriptions.append( description_to_words( train_raw["description"].values[i] ) )

Description 10000 of 49352

Description 20000 of 49352

Description 30000 of 49352



C:\Users\Philipp.Deutsch\AppData\Local\Continuum\Anaconda2\envs\ds\lib\site-packages\bs4\__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Description 40000 of 49352



In [44]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500) 
train_data_features = vectorizer.fit_transform(clean_train_descriptions)
train_data_features = train_data_features.toarray()

In [47]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

vocab = vectorizer.get_feature_names()

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

9259 access
1447 actual
1943 additional
1841 air
1394 alcove
2460 allowed
4651 also
7795 amazing
9098 amenities
2321 ample
2220 anytime
41128 apartment
7119 apartments
20344 appliances
3519 appointment
2168 apt
10433 area
1181 areas
1801 around
4366 art
1375 attended
8140 available
2905 ave
3086 avenue
5882 away
2409 balcony
2287 bar
4993 bars
988 basketball
7949 bath
14114 bathroom
4555 bathrooms
1841 baths
10903 beautiful
2090 beautifully
5816 bed
28941 bedroom
6906 bedrooms
1087 beds
5330 best
1104 bicycle
1776 big
2452 bike
970 billiards
4491 block
4061 blocks
1001 boasts
4639 bond
948 bosch
4561 br
6105 brand
1523 breakfast
3787 brick
3907 bright
1198 broadway
8821 broker
1917 brooklyn
35311 building
1652 buildings
1441 built
1799 bus
1136 buses
1367 cabinet
2530 cabinetry
3601 cabinets
1676 cafes
22730 call
4633 ceiling
10437 ceilings
1225 cell
9077 center
7970 central
1497 charming
1204 chef
1972 chelsea
1268 children
10043 city
1050 classic
1175 clean
1605 cleaning
7158 close
1

In [48]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train_raw["interest_level"].values)

In [50]:
predictions = forest.predict(train_data_features)
ct = pd.crosstab(predictions, train_raw["interest_level"].values, margins=False)
print(ct)

col_0   high    low  medium
row_0                      
high    3142     72     224
low      319  33605    1447
medium   378    607    9558
